In [237]:
import requests
import os
import json
from typing import Dict
# URL to fetch the JSON data
url = "https://csrc.nist.gov/extensions/nudp/services/json/nudp/framework/version/sp_800_171_3_0_0/export/json?element=all"

# File path to store the JSON data
framework_file_path = "../client/public/data/sp_800_171_3_0_0/framework.json"

# Fetch the JSON data
response = requests.get(url)
framework_data = response.json()

In [238]:
withdrawn: Dict[str, str] = {}
for element in framework_data["response"]["elements"]["elements"]:
    if element['element_type'] == "withdraw_reason":
        element_id = element["element_identifier"][-8:]
        withdrawn[element_id] = element["text"]

requirements = [ element for element in framework_data["response"]["elements"]["elements"] if element["element_type"] == "requirement" ]

In [239]:
# File path to store the JSON data
framework_v2_file_path = "../client/public/data/sp_800_171_2_0_0/framework.json"

# Write the JSON data to the file
with open(framework_v2_file_path, 'r') as file:
   v2 = json.load(file)



for v2_element in v2["response"]["elements"]["elements"]:
   identifier = v2_element["element_identifier"][-8:]
   element_type = v2_element["element_type"]
   if identifier in withdrawn:
      if element_type == 'discussion':
         framework_data["response"]["elements"]["elements"].append(v2_element)
      elif element_type == 'requirement':
         for element in framework_data["response"]["elements"]["elements"]:
            if element["element_identifier"][-8:] == identifier:
               element["text"] = v2_element["text"]
               framework_data["response"]["elements"]["elements"].append({
                  **v2_element,
                  "element_identifier": f"SR-{identifier}",
                  "element_type": "security_requirement"
               })
               break

In [240]:
import re
from typing import Dict
from dataclasses import dataclass

pattern = re.compile(r'\d{2}\.\d{2}\.\d{2}')

withdrawn_relations: Dict[str, Dict] = {}

for identifier, text in withdrawn.items():
    matches = pattern.findall(text)
    
    if identifier not in withdrawn_relations:
        withdrawn_relations[identifier] = {
            "withdrawn_into": list(matches),
            "withdrawn_from": list(),
            "value": 0,
            "partial_value": 0,
            "revision": 2,
            "aggregate_value_withdrawn_from": 0,
            "aggregate_partial_value_withdrawn_from": 0
        }
    else:
        withdrawn_relations[identifier]["withdrawn_into"] = withdrawn_relations[identifier]["withdrawn_into"] + list(matches)

    for match in matches:
        if match not in withdrawn_relations:
            withdrawn_relations[match] = {
                "withdrawn_into": list(),
                "withdrawn_from": [identifier],
                "value": 0,
                "partial_value": 0,
                "revision": 2,
                "aggregate_value_withdrawn_from": 0,
                "aggregate_partial_value_withdrawn_from": 0
            }
        else:
            withdrawn_relations[match]["withdrawn_from"].append(identifier)

for element in requirements:
    if element["element_identifier"] not in withdrawn_relations:
        withdrawn_relations[element["element_identifier"]] = {
                "withdrawn_into": list(),
                "withdrawn_from": list(),
                "value": 0,
                "partial_value": 0,
                "revision": 3,
                "aggregate_value_withdrawn_from": 0,
                "aggregate_partial_value_withdrawn_from": 0
        }


In [241]:
%%bash
# Scoring https://www.acq.osd.mil/asda/dpc/cp/cyber/docs/safeguarding/NIST-SP-800-171-Assessment-Methodology-Version-1.2.1-6.24.2020.pdf
mlr --icsv --ojson --jlistwrap cat ../data/values.csv > ../data/values.json

In [242]:
# Write the JSON data to the file
with open("../data/values.json", 'r') as file:
   values = json.load(file)

for item in values:
   identifier = item["identifier"]
   value = item["value"]
   partial_value = item["partial_value"]

   relation = withdrawn_relations[identifier]
   into_count = len(relation["withdrawn_into"])

   if into_count:
      for into_id in relation["withdrawn_into"]:
         withdrawn_relations[into_id]["aggregate_value_withdrawn_from"] += (value / into_count)
         withdrawn_relations[into_id]["aggregate_partial_value_withdrawn_from"] += (partial_value / into_count)

   relation["value"] = value
   relation["partial_value"] = partial_value

with open('../client/public/data/sp_800_171_3_0_0/values.json', 'w') as file:
   json.dump(withdrawn_relations, file, indent=4)

In [243]:
# Ensure the directory exists
os.makedirs(os.path.dirname(framework_file_path), exist_ok=True)

# Write the JSON data to the file
with open(framework_file_path, 'w') as file:
    json.dump(framework_data, file, indent=4)

print(f"JSON data has been stored in {framework_file_path}")

JSON data has been stored in ../client/public/data/sp_800_171_3_0_0/framework.json
